# Library def

In [1]:
import pdb
import collections
import os
import pandas as pd
import re
import requests

from datetime import date, datetime, timedelta
from IPython.display import clear_output
from time import time as timer

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.support import wait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
# Throughout, "att" is short for "archiveTimes table", which contains archive 
# entry info for the date selected in the calendar

In [6]:
# Constants
FEED_URL_STEM = 'https://www.broadcastify.com/listen/feed/'
ARCHIVE_FEED_STEM = 'https://m.broadcastify.com/archives/feed/'
ARCHIVE_DOWNLOAD_STEM = 'https://m.broadcastify.com/archives/id/'
LOGIN_URL = 'https://www.broadcastify.com/login/'
MAIN_URL = 'https://www.broadcastify.com/'

WEBDRIVER_PATH = '../assets/chromedriver'

FILE_REQUEST_WAIT = 3 # seconds
PAGE_REQUEST_WAIT = 1 # seconds

USERNAME = 'cwchiu'
PASSWORD = 'datascientists'

MONTHS = ['','January', 'February', 'March',
      'April', 'May', 'June',
      'July', 'August', 'September',
      'October', 'November', 'December']

# Library-level variables
ArchiveEntry = collections.namedtuple('ArchiveEntry',
                                     'file_uri file_end_datetime mp3_url')
"""
file_uri : str
    The unique ID for an individual archive file, which corresponds to a feed's 
    transmission over a ~30-minute period on a given date. Can be used to find 
    the file's individual download page
file_end_date_time : str
    Date and end time of the transmission in the format YYYYMMDD-HHMM, on a 
    24-hour clock
mp3_url : str
    The URL of the corresponding mp3 file
""";

In [7]:
max_request_wait = max(FILE_REQUEST_WAIT, PAGE_REQUEST_WAIT)

login_data = {
    'username': USERNAME,
    'password': PASSWORD,
    'action': 'auth',
    'redirect': '/'
}

headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) ' +
                  'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/' +
                  '75.0.3770.142 Safari/537.36'
}

login_url = 'https://m.broadcastify.com/login/'

In [8]:
class RequestThrottle:
    def __init__(self):
        self.last_file_req = timer()
        self.last_page_req = timer()
        
    def throttle(self, type='page'):
        if type == 'page':
            while not timer() - self.last_page_req >= PAGE_REQUEST_WAIT:
                pass
#                 print("waiting...")
#                 clear_output(wait=True)
#             print('Throttle clear.')
            self.last_page_req = timer()
        else:
            while not timer() - self.last_file_req >= FILE_REQUEST_WAIT:
                pass
#                 print("waiting...")
#                 clear_output(wait=True)
#             print('Throttle clear.')
            self.last_file_req = timer()        

In [9]:
class NoActiveBrowser(Exception):
    pass

In [20]:
class BroadcastifyArchive:
    def __init__(self, feed_id, username=None, password=None, verbose=1):
        # Attributes
        self.id = feed_id
        self.url = ARCHIVE_FEED_STEM + feed_id
        self.username = username
        self.password = password
        self.entries = [] # list of ArchiveEntry objects
        self.earliest_date = None 
        self.latest_date = None
        self.an = None
        self.verbose = verbose
        
        self.feed_url = FEED_URL_STEM + feed_id

    def build(self, days_back=0): # 0 days back means the active day
        
        all_att_entries = []
        counter = 1
        
        # Make sure days_back is an integer and non-negative
        try:
            days_back = int(days_back)
            if days_back < 0: days_back = 0
            days_back += 1
        except (TypeError):
            raise TypeError("The `days_back` parameter needs an integer between 0 and 180.")
        
        if self.verbose: print('Starting the ArchiveNavigator...')

        # Instantiate the ArchiveNavigator
        self.an = ArchiveNavigator(self.url, self.verbose)
        
        # Add the current (zero-th) day's archiveTimes table entries 
        # (file_uri & file_end_date_time)
        if self.verbose: print(f'Parsing day {counter} of {days_back}: {self.an.active_date}')
        all_att_entries = self.__parse_att(self.an.att_soup)
        self.latest_date = all_att_entries[0][1]
        self.earliest_date = all_att_entries[-1][1]

        # For each day requested...
        for day in range(1, days_back):
            # If clicking the prior day takes us past the beginning of the archive,
            # stop.
            if not self.an.click_prior_day(): break
            
            counter += 1
            if self.verbose: print(f'Parsing day {counter} of {days_back}: {self.an.active_date}')
            
            # Get the archiveTimes table entries (file_uri & file_end_date_time)
            all_att_entries.extend(self.__parse_att(self.an.att_soup))
            self.earliest_date = all_att_entries[-1][1]
            pdb.set_trace()
            
        self.an.close_browser()
        
        # Iterate through att entries to
        ##  - Get the mp3 URL
        ##  - Build an ArchiveEntry, and append to the list
        ## Instantiate the DownloadNavigator
        dn = DownloadNavigator(self.verbose)
        counter = 0
        
        ## Loop & build
        for uri, end_time in all_att_entries:
            counter += 1
            if self.verbose: print(f'Building ArchiveEntry list: {counter} of {len(all_att_entries)}')
            clear_output(wait=True)
            self.entries.append(ArchiveEntry(uri,
                                             end_time,
                                             self.__parse_mp3_path(
                                             dn.get_download_soup(uri))))

        if self.verbose:
            print(f'Archive build complete.')
            print(self)
            
    def download_mp3(self, start_date=-1, end_date=-1, time_of_day=-1):
        pass
    
    def __parse_att(self, att_soup):
        """
        Generates a list of Broadcastify archive file information from
        the `archiveTimes` table on a feed's archive page. Each item in
        the list is a list of two elements:
            - The unique ID for the file, which can be used to find the file's
              individual download page
            - Date and end time of the transmission in the format YYYYMMDD-HHMM,
              on a 24-hour clock

        Parameters
        ----------
        self.soup : bs4.BeautifulSoup
            A BeautifulSoup object containing the feed archive page source code, 
            e.g. from https://m.broadcastify.com/archives/feed/[feed_id]


        """
        
        # Set up a blank list to return
        att_entries = []

        # Loop through all rows of the table
        for row in att_soup.find_all('tr'):    
            # Grab the end time, contained in the row's second <td> tag
            file_end_datetime = self.__get_entry_end_datetime(row.find_all('td')[1].text)
            
            # Grab the file ID
            file_uri = row.find('a')['href'].split('/')[-1]

            # Put the file date/time and URL leaf (as a list) into the list
            att_entries.append([file_uri, file_end_datetime])
        
        return att_entries
    
    def __get_entry_end_datetime(self, time):
        # Better to store the archive entry date/time as datetime and convert to str on export.
        clock = datetime.strptime(time, '%I:%M %p')
        return datetime.combine(self.an.active_date, datetime.time(clock))
        
    def __format_archtimes_date_for_filename(self):
        ### REFACTOR TO HANDLE DATETIME IN ARCHIVEENTRY ###
        # Format the active_date on the ArchiveNavigator as YYYYMMDD
        year = self.an.active_date.year
        month = self.an.active_date.month
        day = self.an.active_date.day
        formatted_date = str(year) + str(month).zfill(2) + str(day).zfill(2)
        
        #             file_end_time = self.__time_to_hhmm(row.find_all('td')[1].text) 

            # Represent the date & end time of the file as YYYYMMDD-HHMM
#             file_end_date_time = '-'.join([archive_date, file_end_time])

        return formatted_date
    
    def __parse_mp3_path(self, download_page_soup):
        # Get the filepath for the mp3 archive
        return download_page_soup.find('a', {'href': re.compile('.mp3')}).attrs['href']

    def __time_to_hhmm(self, s):
        # Converts a string representing a time in HH:MM AM/PM format to a string in 24-hr HHMM
        # More details, since it's a one-line method and this isn't freaking codewars:
            # strptime converts the string to datetime 
                # see https://docs.python.org/2/library/datetime.html#strftime-strptime-behavior
                # and https://stackoverflow.com/questions/19229190/convert-12-hour-into-24-hour-times
            # first split separates YYYY-MM-DD from HH:MM
            # second split gets rid of the colon between HH & MM
            # join puts HHMM together
        return ''.join(str(datetime.strptime(s, '%I:%M %p')).split(' ')[-1].split(':')[:2])

    def __repr__(self):
        return(f'BroadcastifyArchive({len(self.entries)} ArchiveEntries; '
               f'from {str(self.latest_date)} to {str(self.earliest_date)}. '
               f'{self.url})')

In [24]:
class ArchiveNavigator:
    def __init__(self, url, verbose):
        self.url = url
        self.calendar_soup = None
        self.att_soup = None
        self.browser = None
        self.verbose = verbose

        self.active_date = None # currently displayed date
        self.month_max_date = None # latest day in displayed month with archive entries
        self.month_min_date = None # earliest day in displayed month with archive entries
       
        # Get initial page scrape & parse the calendar
        self.open_browser()
        self.__load_nav_page()
        self.__scrape_nav_page()
        self.__parse_calendar()
        
        self.archive_max_date = self.active_date
        
        # https://www.saltycrane.com/blog/2010/10/how-get-date-n-days-ago-python/
        self.archive_min_date = self.archive_max_date - timedelta(days=181)
        
    def click_prior_day(self):
        pdb.set_trace()
        # calculate the prior day
        prior_day = self.active_date - timedelta(days=1)
        
        # would this take us past the archive? if so, stop.
        if prior_day < self.archive_min_date:
            return False
        
        # is the prior day in the previous month? set the xpath class appropriately
        if prior_day < self.month_min_date:
            xpath_class = 'old day'
        else:
            xpath_class = 'day'
        pdb.set_trace()
        xpath_day = prior_day.day
        
        self.__check_browser()
        
        # click the day before the currently displayed day
        calendar_day = self.browser.find_element_by_xpath(
                        f"//td[@class='{xpath_class}' "
                        f"and contains(text(), '{xpath_day}')]")
            # https://stackoverflow.com/questions/2009268/how-to-write-an-xpath-query-to-match-two-attributes
        calendar_day.click()

        # refresh soup & re-parse calendar
        self.__scrape_nav_page()
        self.__parse_calendar()
        
        return self.active_date
    
    def __load_nav_page(self):
        if self.verbose: print('Loading navigation page...')
        self.__check_browser()

        # Browse to feed archive page
        self.browser.get(self.url)   
    
    def __scrape_nav_page(self):
        if self.verbose: print('Scraping navigation page...')
        self.__check_browser()
        
        # Wait for page to render
        element = WebDriverWait(self.browser, 10).until(
                  EC.presence_of_element_located((By.CLASS_NAME, 
                                                  "cursor-link")))

        # Scrape page content
        soup = BeautifulSoup(self.browser.page_source, 'lxml')

        # Isolate the calendar and the archiveTimes table
        self.calendar_soup = soup.find('table', 
                                       {'class': 'table-condensed'})
        self.att_soup = soup.find('table', 
                                  attrs={'id': 'archiveTimes'}
                                  ).find('tbody')
        pdb.set_trace()
        
    def __parse_calendar(self):
        """
        Uses a bs4 ResultSet of the <td> tags representing days currently displayed
        on the calendar to set calendarattributes. Items have the format of 
        `<td class="[class]">[D]</td>` where 
         - [D] is the one- or two-digit day (as a string) and
         - [class] is one of
             "old day"          = a day with archives but in a prior month (clicking
                                  will refresh the calendar)
             "day"              = a past day in the current month
             "active day"       = the day currently displayed in the archiveTimes 
                                  table
             "disabled day"     = a day for which no archive is available in a month
                                  (past or future) that has other days with archives. 
                                  For example, if today is July 27, July 28-31 will 
                                  be disabled days, as will January 1-26 (since the 
                                  archive goes back only 180 days). December 31 would
                                  be an "old disabled day".
                                  past month for which archives are no longer available
             "new disabled day" = a day in a future month
             "old disabled day" = see explanation in "disabled day"
         
        """
        if self.verbose: print('Parsing calendar...')
        
        # Get the tags representing the days currently displayed on the calendar
        days_on_calendar = self.calendar_soup.find_all('td')
        
        # Get the month & year currently displayed
        month, year = self.calendar_soup.find('th', 
                                              {'class': 'datepicker-switch'}
                                              ).text.split(' ')
        
        displayed_month = MONTHS.index(month)
        displayed_year = int(year)
        
        # Parse the various calendar attributes
        active_day = int([day.text for day in days_on_calendar
                           if (day['class'][0] == 'active')][0])
        
        month_max_day = int([day.text for day in days_on_calendar
                              if (day['class'][0] == 'day') or
                                 (day['class'][0] == 'active')][::-1][0])
        
        month_min_day = int(self.__parse_month_min_day(days_on_calendar))
        
        # Set class attributes
        self.active_date = date(displayed_year, displayed_month, active_day)        
        self.month_min_date = date(displayed_year, displayed_month, month_min_day)
        self.month_max_date = date(displayed_year, displayed_month, month_max_day)
        
    def __parse_month_min_day(self, days_on_calendar):
        """Parse the lowest valid day in the displayed month"""
        disabled_found = False
        for day in days_on_calendar:
            if day['class'][0] == 'disabled':
                disabled_found = True
            elif day['class'][0] in 'day active'.split():
                return day.text
            elif day['class'][0] != 'old' and disabled_found:
                return day.text
        
        return None

    def open_browser(self):
        if self.verbose: print('Opening browser...')
        self.browser = webdriver.Chrome('../assets/chromedriver')

    def close_browser(self):
        if self.verbose: print('Closing browser...')
        self.browser.quit()

    def __check_browser(self):
        if not self.browser:
            raise NoActiveBrowser("Please open a browser. And remember to close it when you're done.")
            
    def __repr__(self):
        return(f'ArchiveNavigator(URL: {self.url}, '
               f'Currently Displayed: {str(self.active_date)}, '
               f'Max Day: {str(self.archive_max_date)}, '
               f'Min Day: {str(self.archive_min_date)}, ')

In [14]:
class DownloadNavigator():
    def __init__(self, verbose=0):
        self.download_page_soup = None
        self.current_archive_id = None
        self.verbose = verbose
        self.throttle = RequestThrottle()

    def get_download_soup(self, archive_id):
        self.current_archive_id = archive_id
        
        with requests.Session() as s:
            self.throttle.throttle()
            r = s.post(login_url, data=login_data, headers=headers)

            self.throttle.throttle()
            r = s.get('https://m.broadcastify.com/archives/id/' + archive_id)
            self.download_page_soup = BeautifulSoup(r.text, 'lxml')       

        return self.download_page_soup        
        
    def __repr__(self):
        return(f'DownloadNavigator(Current Archive: {self.current_archive_id})')

In [15]:
# class ArchiveTimesTable:
#     def __init__(self, parent, archive_page_soup):
#         self.parent = parent
#         self.soup = archive_page_soup
#         self.table_entries = self.__parse_entries()
        
#         # Properties
#         @property
#         def table_entries(self):
#             """Username for Broadcastify premium account."""
#             print('Inside property construct.')
#             return self.table_entries
#         @table_entries.setter
#         def table_entries(self, value):
#             self.table_entries = value
#             print('Inside property construct.')
        
# #     def __parse_entries(self):
# #         """
# #         Generates a list of Broadcastify archive file information from
# #         the `archiveTimes` table on a feed's archive page. Each item in
# #         the list is a list of two elements:
# #             - The unique ID for the file, which can be used to find the file's
# #               individual download page
# #             - Date and end time of the transmission in the format YYYYMMDD-HHMM,
# #               on a 24-hour clock

# #         Parameters
# #         ----------
# #         self.soup : bs4.BeautifulSoup
# #             A BeautifulSoup object containing the feed archive page source code, 
# #             e.g. from https://m.broadcastify.com/archives/feed/[feed_id]


# #         """
# #         # Set up a blank list to return
# #         table_entry_builder = []

# #         # Isolate the `archive_times` table body
# #         archive_times = self.soup.find('table', attrs={'id': 'archiveTimes'}).find('tbody')

# #         # Find the date of transmission of the archived files
# #         archive_date = self.__format_archive_date()

# #         # Loop through all rows of the table
# #         for row in archive_times.find_all('tr'):

# #             # Grab the end time, contained in the row's second <td> tag
# #             file_end_time = self.__time_to_hhmm(row.find_all('td')[1].text) 

# #             # Represent the date & end time of the file as YYYYMMDD-HHMM
# #             file_end_date_time = '-'.join([archive_date, file_end_time])

# #             # Grab the file ID
# #             file_uri = row.find('a')['href'].split('/')[-1]

# #             # Put the file date/time and URL leaf (as a list) into the list
# #             table_entry_builder.append([file_uri, file_end_date_time])
        
# #         return table_entry_builder

#     def __get_mp3_urls(self):
#         # Get the first page
#         parent.last_page_request_time = timer()
#         browser.get('https://m.broadcastify.com/archives/id/' + self.table_entries[0][1])


# #         # Log in so we can download files
# #         ## Store the fields for username + password
# #         username_field = browser.find_element_by_id("signinSrEmail") 
# #         password_field = browser.find_element_by_id("signinSrPassword")

# #         ## Type username + password, and hit "enter"
# #         username_field.send_keys(USERNAME)
# #         password_field.send_keys(PASSWORD)
# #         password_field.send_keys(Keys.RETURN)

# #         ## Wait for login to complete
# #         browser.implicitly_wait(2)

#         # Get the filepath for the mp3 archive
#         self.table_entries[0].append(get_mp3_path(BeautifulSoup(browser.page_source, 'lxml')))

#         for row in self.table_entries[1:11]:
#            # Wait until some time has passed, out of courtesy
#             while not courtesy_wait(parent.last_page_request_time): pass

#             # Get the next archive page, recording the time
#             browser.get('https://m.broadcastify.com/archives/id/' + row[1])
#             parent.last_page_request_time = timer()

#             # Get the filepath for the mp3 archive
#             row.append(get_mp3_path(BeautifulSoup(browser.page_source, 'lxml')))
        
#     def __format_archive_date(self):

#         # Extract the day, month, and year of the data displayed on the page
#         day = self.soup.find('td', {'class': 'active day'}).text
#         month, year = self.soup.find('th', {'class': 'datepicker-switch'}).text.split()

#         # Format the date as YYYYMMDD
#         formatted_date = str(year) + str(MONTHS.index(month)).zfill(2) + day.zfill(2)

#         return formatted_date
    
#     def __time_to_hhmm(self, s):
#         # More details, since it's a one-line method and this isn't freaking codewars:
#             # strptime converts the string to datetime 
#                 # see https://docs.python.org/2/library/datetime.html#strftime-strptime-behavior
#                 # and https://stackoverflow.com/questions/19229190/convert-12-hour-into-24-hour-times
#             # first split separates YYYY-MM-DD from HH:MM
#             # second split gets rid of the colon between HH & MM
#             # join puts HHMM together
#         # Converts a string representing a time in HH:MM AM/PM format to a string in 24-hr HHMM
#         return ''.join(str(datetime.strptime(s, '%I:%M %p')).split(' ')[-1].split(':')[:2])
            
#     def __repr__(self):
#         return (f'ArchiveTimesTable({len(self.table_entries)} entries)')

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

# Test code

In [16]:
DATA_PATH = '../data/'
TEST_MP3_OUT_PATH = DATA_PATH + 'test_data/test_mp3/'

# TEST_FEED_ID = '18812'
TEST_FEED_ID = '10904'
TEST_DOWNLOAD_ID = '774426456'

In [22]:
archive = BroadcastifyArchive(TEST_FEED_ID, USERNAME, PASSWORD)

In [25]:
archive.build(days_back=1)

Starting the ArchiveNavigator...
Opening browser...
Loading navigation page...
Scraping navigation page...
--Return--
> <ipython-input-24-688aa049b9f2>(81)__scrape_nav_page()->None
-> pdb.set_trace()
(Pdb) continue
Parsing calendar...
Parsing day 1 of 2: 2019-07-30
> <ipython-input-24-688aa049b9f2>(27)click_prior_day()
-> prior_day = self.active_date - timedelta(days=1)
(Pdb) s
> <ipython-input-24-688aa049b9f2>(30)click_prior_day()
-> if prior_day < self.archive_min_date:
(Pdb) prior_day
datetime.date(2019, 7, 29)
(Pdb) self.archive_min_date
datetime.date(2019, 1, 30)
(Pdb) s
> <ipython-input-24-688aa049b9f2>(34)click_prior_day()
-> if prior_day < self.month_min_date:
(Pdb) s
> <ipython-input-24-688aa049b9f2>(37)click_prior_day()
-> xpath_class = 'day'
(Pdb) self.month_min_date
datetime.date(2019, 7, 1)
(Pdb) s
> <ipython-input-24-688aa049b9f2>(38)click_prior_day()
-> pdb.set_trace()
(Pdb) s
--Call--
> /anaconda3/lib/python3.7/pdb.py(1606)set_trace()
-> def set_trace(*, header=None):
(

(Pdb) exit


BdbQuit: 

In [ ]:
archive.entries